## The Battle of Neighborhoods: London

### Clustering schools based on house price and the number of libraries and museums

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd 
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# import matplotlib.colors as colors

# from sklearn.cluster import KMeans

import folium

from time import sleep

## Data preparation

### London boroughs

In [2]:
# read the information of London borough from wiki
url='https://en.wikipedia.org/wiki/List_of_London_boroughs'
London_data=pd.read_html(url)

In [3]:
# get the info of 32 Boroughs and the City of London
London_data[:2]

[                            Borough     Inner Status  \
 0     Barking and Dagenham [note 1]       NaN    NaN   
 1                            Barnet       NaN    NaN   
 2                            Bexley       NaN    NaN   
 3                             Brent       NaN    NaN   
 4                           Bromley       NaN    NaN   
 5                            Camden       NaN    NaN   
 6                           Croydon       NaN    NaN   
 7                            Ealing       NaN    NaN   
 8                           Enfield       NaN    NaN   
 9                Greenwich [note 2]  [note 3]  Royal   
 10                          Hackney       NaN    NaN   
 11  Hammersmith and Fulham [note 4]       NaN    NaN   
 12                         Haringey  [note 3]    NaN   
 13                           Harrow       NaN    NaN   
 14                         Havering       NaN    NaN   
 15                       Hillingdon       NaN    NaN   
 16                         Hou

In [4]:
# Convert to dataframe 
Borough=pd.DataFrame(London_data[0])
City_of_London=pd.DataFrame(London_data[1])

# add city of London to Borough for convenience
Greater_London=Borough.append(City_of_London)

# reset index
Greater_London.reset_index(drop=True, inplace=True)

Greater_London.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map,Population(2011 est),Nr. inmap
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352.0,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25.0,NaN,NaN
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088.0,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31.0,NaN,NaN
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687.0,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23.0,NaN,NaN
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264.0,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12.0,NaN,NaN
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899.0,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20.0,NaN,NaN


__clean data__

In [9]:
# get the list of boroughs
london_borough=Greater_London[['Borough']]

# edit the names of Borough, delete suffix [note #]
# get borough names with suffix
borough_with_suffix=london_borough['Borough'].str.contains('note')
for i, borough in london_borough[borough_with_suffix].iterrows():
    name_no_suffix=borough['Borough'][:-9]
    london_borough.loc[i,'Borough']=name_no_suffix

london_borough.head()

/Users/wangyifei/opt/anaconda3/envs/Cousera_learning/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/wangyifei/opt/anaconda3/envs/Cousera_learning/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Borough
0,Barking and Dagenham
1,Barnet
2,Bexley
3,Brent
4,Bromley


In [10]:
# Gathering the Latitude and Longitude coordinates for each borough using geolocater
for i in range(london_borough.shape[0]):
    borough=london_borough.loc[i,'Borough']
    address= borough + ',England'
    geolocator = Nominatim(user_agent="LDN_explorer")
    
    location = geolocator.geocode(address)
    
    latitude = location.latitude
    longitude = location.longitude        
    london_borough.loc[i,'Latitude']=latitude
    london_borough.loc[i,'Longitude']=longitude
    sleep(1)

london_borough.head()

/Users/wangyifei/opt/anaconda3/envs/Cousera_learning/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/wangyifei/opt/anaconda3/envs/Cousera_learning/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Borough,Latitude,Longitude
0,Barking and Dagenham,51.554117,0.150504
1,Barnet,51.653090,-0.200226
2,Bexley,51.441679,0.150488
3,Brent,51.441635,0.234519
4,Bromley,51.402805,0.014814


In [57]:
# save file
london_borough.to_csv('London_borough_location.csv',index=False)

### House Price by Borough

In [11]:
filename='London-house-prices-borough.csv'

In [12]:
house_price=pd.read_csv(filename)
house_price.head()

,Code,Area,Year,Measure,Value
0,E09000001,City of London,Year ending Dec 1995,Median,"105,000"
1,E09000002,Barking and Dagenham,Year ending Dec 1995,Median,"49,000"
2,E09000003,Barnet,Year ending Dec 1995,Median,"85,125"
3,E09000004,Bexley,Year ending Dec 1995,Median,"62,000"
4,E09000005,Brent,Year ending Dec 1995,Median,"68,000"


In [13]:
house_price.shape

(12015, 5)

In [14]:
house_price.groupby('Measure').count()

,Code,Area,Year,Value
Measure,,,,
Mean,4005,4005,4005,4005
Median,4005,4005,4005,4005
Sales,4005,4005,4005,4005


We will use the Median and Mean house price at the end of each year from 1995 to 2017 for modeling and predicting

In [47]:
# select the year-end data
house_price_dec=house_price.loc[house_price['Year'].str.contains('Dec'),:]
house_price_dec.head()

,Code,Area,Year,Measure,Value
0,E09000001,City of London,Year ending Dec 1995,Median,"105,000"
1,E09000002,Barking and Dagenham,Year ending Dec 1995,Median,"49,000"
2,E09000003,Barnet,Year ending Dec 1995,Median,"85,125"
3,E09000004,Bexley,Year ending Dec 1995,Median,"62,000"
4,E09000005,Brent,Year ending Dec 1995,Median,"68,000"


In [48]:
# set column 'Year' to type int
Year=house_price_dec['Year'].str.split(' ',expand = True)

house_price_dec['YearEnd']=Year[3]

house_price_dec.head()

/Users/wangyifei/opt/anaconda3/envs/Cousera_learning/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Code,Area,Year,Measure,Value,YearEnd
0,E09000001,City of London,Year ending Dec 1995,Median,"105,000",1995
1,E09000002,Barking and Dagenham,Year ending Dec 1995,Median,"49,000",1995
2,E09000003,Barnet,Year ending Dec 1995,Median,"85,125",1995
3,E09000004,Bexley,Year ending Dec 1995,Median,"62,000",1995
4,E09000005,Brent,Year ending Dec 1995,Median,"68,000",1995


In [49]:
# drop columns
house_price_dec.drop(columns=['Code','Year'],inplace=True)

# rename columns
house_price_dec.rename(columns={'Area':'Borough','Value':'Price'},inplace=True)

house_price_dec=house_price_dec.reset_index(drop=True)

/Users/wangyifei/opt/anaconda3/envs/Cousera_learning/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/wangyifei/opt/anaconda3/envs/Cousera_learning/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [50]:
house_price_dec.head()

,Borough,Measure,Price,YearEnd
0,City of London,Median,"105,000",1995
1,Barking and Dagenham,Median,"49,000",1995
2,Barnet,Median,"85,125",1995
3,Bexley,Median,"62,000",1995
4,Brent,Median,"68,000",1995


In [53]:
# drop rows with borough not in the london_borough dataframe
local_index=[]
for index in range(0,house_price_dec.shape[0]):
    if house_price_dec['Borough'].iloc[index] not in london_borough['Borough'].tolist():
        local_index.append(index)

house_price_dec=house_price_dec.drop(local_index,axis=0)
house_price_dec.reset_index(drop=True, inplace=True)

print(house_price_dec.shape)
house_price_dec.head()

(2277, 4)


,Borough,Measure,Price,YearEnd
0,City of London,Median,"105,000",1995
1,Barking and Dagenham,Median,"49,000",1995
2,Barnet,Median,"85,125",1995
3,Bexley,Median,"62,000",1995
4,Brent,Median,"68,000",1995


In [54]:
# select median and mean value
house_price_median=house_price_dec.loc[house_price_dec['Measure']=='Median',:]
house_price_mean=house_price_dec.loc[house_price_dec['Measure']=='Mean',:].reset_index(drop=True)

In [55]:
house_price_median.head()

,Borough,Measure,Price,YearEnd
0,City of London,Median,"105,000",1995
1,Barking and Dagenham,Median,"49,000",1995
2,Barnet,Median,"85,125",1995
3,Bexley,Median,"62,000",1995
4,Brent,Median,"68,000",1995


In [56]:
print(house_price_mean.shape)
house_price_mean.head()

(759, 4)


,Borough,Measure,Price,YearEnd
0,City of London,Mean,"146,043",1995
1,Barking and Dagenham,Mean,"50,615",1995
2,Barnet,Mean,"111,019",1995
3,Bexley,Mean,"66,294",1995
4,Brent,Mean,"77,258",1995


In [58]:
# save file
house_price_median.to_csv('London_MedianHousePrice_borough_1995_2017.csv',index=False)
house_price_mean.to_csv('London_MeanHousePrice_borough_1995_2017.csv',index=False)


### Schools Atlas

In [22]:
file_school='all_schools.csv'
school=pd.read_csv(file_school)

In [23]:
school.head()

,OBJECTID,URN,SCHOOL_NAM,TYPE,PHASE,ADDRESS,TOWN,POSTCODE,STATUS,GENDER,...,LA_NAME,WEBLINK,AGE,map_icon,NEW_URN,OLD_URN,map_icon_l,Primary,x,y
0,1,135155,Ayesha Siddiqa Girls School,Other Independent School,Not applicable,165-169 The Broadway,Southall,UB1 1LR,Open,Girls,...,Ealing,,19-Nov,,,,2,0,-0.378496,51.507500
1,2,140492,Beis Medrash Elyon,Other Independent School,Not applicable,233 West Hendon Broadway,London,NW9 7DG,Open,Boys,...,Barnet,,16-Nov,,,,2,0,-0.241628,51.578999
2,3,141411,Big Creative Independent School,Other Independent School,Not applicable,Silver Birch House,Walthamstow,E17 5SD,Open,Mixed,...,Waltham Forest,,15 - 16,,,,2,0,-0.042590,51.594002
3,4,142336,Wetherby Senior School,Other Independent School,Not applicable,100 Marylebone Lane,London,W1U 2QB,Open,Boys,...,Westminster,,16-Nov,,,,2,0,-0.150409,51.517601
4,5,100042,St Mary's Kilburn Church of England Primary Sc...,Voluntary Aided School,Primary,Quex Road,London,NW6 4PG,Open,Mixed,...,Camden,http://www.stmarykilburn.camden.sch.uk/,11-Mar,VOLUNTARY,,,2,1,-0.193367,51.540402


In [24]:
# select name, addresses, boroughs, towns, postcodes and location coordinates as school_data
school_data=school[['SCHOOL_NAM', 'ADDRESS', 'TOWN', 'LA_NAME','POSTCODE']]

school_data['Latitude']=school[['y']]
school_data['Longitude']=school[['x']]

print(school_data.shape)
school_data.head()



(3889, 7)


/Users/wangyifei/opt/anaconda3/envs/Cousera_learning/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/wangyifei/opt/anaconda3/envs/Cousera_learning/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,SCHOOL_NAM,ADDRESS,TOWN,LA_NAME,POSTCODE,Latitude,Longitude
0,Ayesha Siddiqa Girls School,165-169 The Broadway,Southall,Ealing,UB1 1LR,51.507500,-0.378496
1,Beis Medrash Elyon,233 West Hendon Broadway,London,Barnet,NW9 7DG,51.578999,-0.241628
2,Big Creative Independent School,Silver Birch House,Walthamstow,Waltham Forest,E17 5SD,51.594002,-0.042590
3,Wetherby Senior School,100 Marylebone Lane,London,Westminster,W1U 2QB,51.517601,-0.150409
4,St Mary's Kilburn Church of England Primary Sc...,Quex Road,London,Camden,NW6 4PG,51.540402,-0.193367


In [28]:
# drop rows with boroughs that are not in the london_borough dataframe
local_index = []
for index in range(0,school_data.shape[0]):
    if school_data['LA_NAME'].iloc[index] not in london_borough['Borough'].tolist():
        local_index.append(index)

school_data=school_data.drop(local_index, axis=0)
school_data.reset_index(drop=True, inplace=True)

print(school_data.shape)
school_data.head()


(3372, 7)


,SCHOOL_NAM,ADDRESS,TOWN,LA_NAME,POSTCODE,Latitude,Longitude
0,Ayesha Siddiqa Girls School,165-169 The Broadway,Southall,Ealing,UB1 1LR,51.507500,-0.378496
1,Beis Medrash Elyon,233 West Hendon Broadway,London,Barnet,NW9 7DG,51.578999,-0.241628
2,Big Creative Independent School,Silver Birch House,Walthamstow,Waltham Forest,E17 5SD,51.594002,-0.042590
3,Wetherby Senior School,100 Marylebone Lane,London,Westminster,W1U 2QB,51.517601,-0.150409
4,St Mary's Kilburn Church of England Primary Sc...,Quex Road,London,Camden,NW6 4PG,51.540402,-0.193367


In [59]:
# save file
school_data.to_csv('London_school_location.csv',index=False)


__Load Data for analysis__